In [ ]:
!tar xvzf ../input/200-bird-species-with-11788-images/CUB_200_2011.tgz

In [ ]:
!head -70 CUB_200_2011/README

In [ ]:
import tracemalloc
tracemalloc.start()

import sys
import time
import os
import gc
import re
import csv
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
tf.data.experimental.enable_debug_mode()
import tensorflow_datasets as tfds
import cv2
import sklearn

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

assert len(tf.config.experimental.list_physical_devices('GPU')) > 0, "No GPU found. Try to connect one..." #https://github.com/BaranovMykola/Tensorflow2Snippets/blob/master/image_dataset.ipynb

In [ ]:
ROOT = "./CUB_200_2011"
IMAGES = f"{ROOT}/images"
CHANNEL_NUM = 3
EXAMPLES_IN_CUB200 = 11788
#
IMAGE_SIDE = 224
NUM_CLASSES = 60
# CLASSES_OFFSET = 0
BATCH_SIZE = 64

##################################################

def process_image(file_name): #https://github.com/BaranovMykola/Tensorflow2Snippets/blob/master/image_dataset.ipynb
    image = tf.io.read_file(tf.strings.join((IMAGES, file_name), separator="/"))
    image = tf.image.decode_jpeg(contents=image, channels=3)
    image = tf.image.resize(image, (IMAGE_SIDE,)*2)
    image /= 255.0
    return image

##################################################
#Read Files To pd.DataFrame
classes            = pd.read_csv(f"{ROOT}/classes.txt",            sep=" ", names=["class_id", "class_name"],        index_col="class_id")
classes            = list(classes["class_name"])
#
images             = pd.read_csv(f"{ROOT}/images.txt",             sep=" ", names=["image_id", "path"],              index_col="image_id")
train_test         = pd.read_csv(f"{ROOT}/train_test_split.txt",   sep=" ", names=["image_id", "is_training_image"], index_col="image_id").applymap(bool)
image_class_labels = pd.read_csv(f"{ROOT}/image_class_labels.txt", sep=" ", names=["image_id", "class_id"],          index_col="image_id")
##############################

train_test_ds = tf.data.Dataset.zip((
    tf.data.Dataset.from_tensor_slices(train_test.is_training_image.to_numpy()), #Get IS_TRAIN indicator
    tf.data.Dataset.from_tensor_slices(images.path.to_numpy()).map(process_image, num_parallel_calls=tf.data.AUTOTUNE), #Get paths and map them
    tf.data.Dataset.from_tensor_slices(image_class_labels.class_id.to_numpy()) #Get labels
)).filter(lambda is_train, image, label: label <= NUM_CLASSES) \
  .map(lambda is_train, image, label: (is_train, image, tf.one_hot(label-1, depth=NUM_CLASSES)), num_parallel_calls=tf.data.AUTOTUNE) #filter : get CLASS_NUM classes from some widow of with CLASSES_WINDOW_SELECTION_OFFSET; map : Subtract 1 and WINDOW_WIDTH to make indexes start with 0, and use One Hot

train = train_test_ds \
    .filter(lambda is_train, *_ : is_train) \
    .map(lambda *x: x[1:], num_parallel_calls=tf.data.AUTOTUNE) \
    .shuffle(buffer_size=int(EXAMPLES_IN_CUB200/10)) \
    .batch(BATCH_SIZE, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE) \
    .cache() \
    .prefetch(tf.data.AUTOTUNE)

test = train_test_ds \
    .filter(lambda is_train, *_ : not is_train) \
    .map(lambda *x: x[1:], num_parallel_calls=tf.data.AUTOTUNE) \
    .batch(BATCH_SIZE, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE) \
    .cache() \
    .prefetch(tf.data.AUTOTUNE)

In [ ]:
# l = 0.001

inputs = tf.keras.layers.Input(shape=(IMAGE_SIDE, IMAGE_SIDE, 3))
#Augmentation
data_aug = tf.keras.Sequential([
    tf.keras.layers.RandomContrast(factor=.2),
    tf.keras.layers.RandomFlip(mode="horizontal"),
    tf.keras.layers.RandomRotation(factor=.12),
    tf.keras.layers.RandomHeight(factor=(.25, .25)),
    tf.keras.layers.RandomWidth(factor=(.25, .25)),
    tf.keras.layers.RandomCrop(height=IMAGE_SIDE, width=IMAGE_SIDE),
])

#Encoder
encoder = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(IMAGE_SIDE, IMAGE_SIDE, 3),
    pooling=None,
)
# encoder = tf.keras.applications.vgg16.VGG16(
#     include_top=False,
#     weights='imagenet',
#     input_shape=(IMAGE_SIDE, IMAGE_SIDE, 3),
#     pooling=None,
# )
# encoder = tf.keras.applications.resnet50.ResNet50(
#     include_top=False,
#     weights='imagenet',
#     input_shape=(IMAGE_SIDE, IMAGE_SIDE, 3),
#     pooling=None,
# )
encoder.trainable=False
x = encoder(inputs)

#Experts  
es = []
F = 64
for i in range(int(NUM_CLASSES)):
    #Shortcut
    sc = tf.keras.layers.Conv2D(filters=F, kernel_size=(1, 1), strides=(1, 1), padding="same")(x)
    sc = tf.keras.layers.BatchNormalization(axis = 3)(sc)
    
    e = tf.keras.layers.Conv2D(filters=F, kernel_size=(3, 3), padding="same")(x)
    e = tf.keras.layers.BatchNormalization(axis = 3)(e)
    e = tf.keras.layers.Activation('relu')(e)
    
    e = tf.keras.layers.Conv2D(filters=F, kernel_size=(3, 3), padding="same")(e)
    e = tf.keras.layers.BatchNormalization(axis = 3)(e)
    e = tf.keras.layers.Activation('relu')(e)
    
    e = tf.keras.layers.Conv2D(filters=F, kernel_size=(3, 3), padding="same")(e) 
    e = tf.keras.layers.BatchNormalization(axis = 3)(e)
    e = tf.keras.layers.Add()((e,sc))
    e = tf.keras.layers.Activation("relu")(e)
    es.append(e)

In [ ]:
x = tf.keras.layers.Concatenate(axis=2)(es)
size_n_stride = x.shape[-3]
x = tf.keras.layers.Conv2D(filters=1, kernel_size=(size_n_stride, size_n_stride), strides=(size_n_stride, size_n_stride), activation=None)(x)
base = tf.keras.Model(inputs=inputs, outputs=x)

# #Model
model = tf.keras.Sequential([
    data_aug,
    base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Activation("sigmoid")
])
# model = tf.keras.Sequential([
#     data_aug,
#     base,
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=NUM_CLASSES, activation="sigmoid")
# ])

In [ ]:
#Compilation
steps = int(EXAMPLES_IN_CUB200/BATCH_SIZE*0.5*NUM_CLASSES/200) #Enough steps to decrease lr each epoch by decay_rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=steps,
    decay_rate=0.95
)
print(f"decay steps : {steps}")

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32)
    return lr

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
lr_metric = get_lr_metric(optimizer)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', lr_metric])

In [ ]:
start_time = time.time()
history = model.fit(train, epochs=96, validation_data=test)
end_time = time.time()
print(f"Program execution time : {end_time-start_time}")

In [ ]:
print(f"acc : {history.history['accuracy'][-1]}; val_acc : {history.history['val_accuracy'][-1]}")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(history.history["accuracy"], label="accuracy")
ax.plot(history.history["val_accuracy"], label="val_accuracy")
ax.set_xlabel("epoch")
ax.set_ylabel("accuracy")
ax.legend()

In [ ]:
with open('/history.pickle', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)